## Imports and declerations

In [2]:
import pickle
import os
import string
import re
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import copy
from nltk.stem import PorterStemmer
InteractiveShell.ast_node_interactivity = "all"

## Data Structure:

In [3]:
class PostingList(object):
    def __init__(self):
        self.total_count = 0
        self.token = ''
        self.occurrance = {
#             'doc_id':0 = 'positions' : [],
#             
        }
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
         
    def __repr__(self):
        
        return f'total_cnt : {self.total_count} docs : [{self.occurrance.keys()}]'
    def __len__(self):
        return len(self.occurrance)
    def addOccurrance(self, doc_id, position):
        self.total_count += 1
#         print(position)
        if doc_id not in self.occurrance.keys():
            self.occurrance[doc_id] = []
        self.occurrance[doc_id].append(position)
#         self.occurrance[doc_id]['position'].append(pos)
    
class InvertedIndex(object):
    
    def __init__(self):
        self.index = {}
        self.docs = {}

    def get_term_postings(self, term):
        if term in self.index.keys():
            return self.index[term]
        else:
            raise ValueError(f'{term} is not present in Index. Plese Try Again')
            return PostingList()
    
    def __len__(self):
        return len(self.index.keys())

## Helper Functions:

In [4]:
# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))

# Clean Query Term
def clean_word(word):
    # Case Folding
    word = word.lower()
     # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
#     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
#     print(word)
    word = ps.stem(word)
#     print(word)
    return word

## Term - Document Indexing:

In [5]:
vocab = set()
doc_contents = []
inverted_index = InvertedIndex()
printable = set(string.printable) 
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


ps = PorterStemmer()

stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

    
    
for file_number in range(0, 56):
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}
        
#         {
#             'total_count' : 0,
#             'postings' : {
#                 'count':0,
#                 'doc_id':0,
#                 'positions':[]
#             }
#         }
        
        for line_no,line in enumerate(lines):
            # Skip Heading Line
            if line_no == 0:
                continue
            doc_set = set()
            # split words at . , whitespace ? ! : ;
            position['row'] = line_no 
            position['col'] = 0
            for word in re.split('[.\s,?!:;-]', line):
                position['col'] += len(word) + 1
                position['token_no'] += 1
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                
                
                
                word = ps.stem(word)
                    
                vocab.add(word)
                
                doc_set.add(word)
                
                if word in inverted_index.index.keys():
                    
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position)) 
                else:
                    plist = PostingList()
                    inverted_index.index[word] = plist
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position))
                    
        inverted_index.docs[file_number] = doc_set
        doc_contents.append(doc_set)

In [6]:
print('Total Vocabulary Size ')
print(len(inverted_index.index.keys()))
print('Total Number of Documents ')
print(len(inverted_index.docs))
# print('Occurances of query: hammer')
# inverted_index.index[ps.stem('hammer')].occurrance

Total Vocabulary Size 
4781
Total Number of Documents 
56


In [7]:
# To save Inverted Index in File
with open('pickled/inverted_index.p', 'wb') as index_file:
    pickle.dump(inverted_index, index_file)

In [8]:
# To Load Inverted Index from File
with open('pickled/inverted_index.p', 'rb') as index_file:
    inverted_index = pickle.load(index_file)

In [9]:
def intersect_posting(p1, p2):
    if len(p1) == 0 or len(p2) == 0:
        return PostingList()

    if isinstance(p1, set) and isinstance(p2, set):
        return p1.intersection(p2)
    elif isinstance(p1, set):
        return p1.intersection(p2.occurrance.keys())
    elif isinstance(p2, set):
        return p2.intersection(p1.occurrance.keys())
    pn = PostingList()
    # pn.token = f'{p1.token} & {p2.token}'
    for pn_keys in (p1.occurrance.keys() & p2.occurrance.keys()) :
        pn.addOccurrance(pn_keys, p1.occurrance[pn_keys])
        pn.addOccurrance(pn_keys, p2.occurrance[pn_keys])
    return pn


def union_posting(p1, p2):
    if len(p1) == 0:
        return p2
    elif len(p2) == 0:
        return p1
    if isinstance(p1, set) and isinstance(p2, set):
        return p1.union(p2)
    elif isinstance(p1, set):
        return p1.union(p2.occurrance.keys())
    elif isinstance(p2, set):
        return p2.union(p1.occurance.keys())
    
    pn = PostingList()
    # pn.token = f'{p1.token} | {p2.token}'
    for pn1_keys in p1.occurrance.keys() :
        pn.addOccurrance(pn1_keys, p1.occurrance[pn1_keys])
    for pn2_keys in p2.occurrance.keys() :
        pn.addOccurrance(pn2_keys, p2.occurrance[pn2_keys])
    
    return pn

def inverse_posting(inverted_index,p):
    print(p)
    if isinstance(p, set) :
        print('Returning ')
        print(set(inverted_index.docs).difference(p))
        return set(inverted_index.docs).difference(p)
    else:
        print(set(inverted_index.docs).difference(set(p.occurrance.keys())))
        return set(inverted_index.docs).difference(set(p.occurrance.keys()))
    return inverted_index.docs.keys() - p.occurrance.keys()

In [10]:
    
def build_index():
    path_to_data = os.path.dirname(__file__) + '../../data/'
    print(os.path.dirname(__file__))
    print(path_to_data)
    vocab = set()
    doc_contents = []
    inverted_index = InvertedIndex()
    printable = set(string.printable) 
    # Printable characters are
    # 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
    # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


    ps = PorterStemmer()
    stop_words = set()
    with open(path_to_data+'Stopword-List.txt', 'r') as stop_word_file:
        lines = stop_word_file.readlines()
        for line in lines:
            stop_words.add(line.split('\n')[0])
        stop_words.remove('')
    print(stop_words)

    for file_number in range(0, 56):
        with open(path_to_data + f'Trump Speechs/speech_{file_number}.txt', 'r') as file1:
            lines = file1.readlines()
            print(f'File Number : speech_{file_number}.txt' )
            print(lines[0])
            position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}

            for line_no,line in enumerate(lines):
                doc_set = set()
                # split words at . , whitespace ? ! : ;
                position['row'] = line_no 
                position['col'] = 0
                for word in re.split('[.\s,?!:;-]', line):
                    position['col'] += len(word) + 1
                    position['token_no'] += 1
                    # Case Folding
                    word = word.lower()
                    
                    # Filter non-ASCII characters
                    word = ''.join(filter(lambda x: x in printable, word))
                    
                    # Remove Punctuations
                    word = remove_punctuation(word)
                    
                    if re.match('\d+[A-Za-z]+',word):
                        word = re.split('\d+',word)[1]
                    if re.match('[A-Za-z]+\d+',word):
                        word = re.split('\d+',word)[0]
                    
                    if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                        continue
                    if word in stop_words:
                        continue

                    word = ps.stem(word)
                        
                    vocab.add(word)
                    
                    doc_set.add(word)
                    
                    if word in inverted_index.index.keys():
                        
                        inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position)) 
                    else:
                        plist = PostingList()
                        inverted_index.index[word] = plist
                        inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position))
                        
            inverted_index.docs[file_number] = doc_set
            doc_contents.append(doc_set)
    ii = InvertedIndexModel()
    ii.status = True
    ii.data = inverted_index
    ii.save()
    return True


boperators = ['and', 'or']
uoperators = ['not']

# Token types
#
# EOF (end-of-file) token is used to indicate that
# there is no more input left for lexical analysis
LPAREN, RPAREN, EOF, TERM, AND, OR, NOT = (
    '(', ')', 'EOF', 'TERM', 'AND','OR', 'NOT'
)


class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
        self.inverse = False
        self.row = []

    def __str__(self):
        """String representation of the class instance.

        Examples:
            Token(TERM, Hello)
            Token(AND, '&')
            Token(NOT, '!')
        """
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()


class Lexer(object):
    def __init__(self, text):
        # client string input, e.g. "hello | world & (why | are | you)"
        self.text = text
        # self.pos is an index into self.text
        self.pos = 0
        self.current_char = self.text[self.pos]
        

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        """Advance the `pos` pointer and set the `current_char` variable."""
        self.pos += 1
        if self.pos > len(self.text) - 1:
            self.current_char = None  # Indicates end of input
        else:
            self.current_char = self.text[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):

        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)
    def word(self):
        result = ''
        # while self.current_char is not None and (self.current_char.isalpha() or self.current_char == '_'):
        while self.current_char is not None and (self.current_char in printable) and (self.current_char not in (' ', '|','&','!', '(', ')')):
            result += self.current_char
            self.advance()
        return str(result)

    def get_next_token(self):
        """Lexical analyzer (also known as scanner or tokenizer)

        This method is responsible for breaking a sentence
        apart into tokens. One token at a time.
        """
        while self.current_char is not None:

            if self.current_char.isspace():
                self.skip_whitespace()
                continue
                        
            if self.current_char == '&':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(AND, 'AND')
            
            if self.current_char == '|':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(OR, 'OR')
            
            if self.current_char == '!':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(NOT,'NOT')
            

            if self.current_char == '(':
                self.advance()
                return Token(LPAREN, '(')

            if self.current_char == ')':
                self.advance()
                return Token(RPAREN, ')')
            
            if self.current_char.isalpha():      
#                 print('Got token  ' + self.current_char)
                return Token(TERM, self.word())
            
            

            self.error()

        return Token(EOF, None)

class AST(object):
    pass


class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right
        self.inverse = False
        self.row = []
        self.value = ''
        
class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
        self.inverse = False
        self.row = []

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        
        
        if token.type == TERM:
            self.eat(TERM)
            return Num(token)
        
        elif token.type == NOT:
            self.eat(NOT)
            node = self.expr()
            node.inverse = True
            return node
            
        
        
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        
        node = self.factor()

        while self.current_token.type in (AND,):
            token = self.current_token
         
            if token.type == AND:
                self.eat(AND)
            
            node = BinOp(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        node = self.term()

        while self.current_token.type in (OR,):
            token = self.current_token
            if token.type == OR:
                self.eat(OR)
            
            node = BinOp(left=node, op=token, right=self.term())

        return node

    def parse(self):
        return self.expr()


class NodeVisitor(object):
    def visit(self, node):
#         print('Checking Node Name')
        
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__name__))


class Interpreter(NodeVisitor):
    def __init__(self, parser, index, ps):
        self.parser = parser
        self.index = index
        self.ps = ps

    def visit_BinOp(self, node):
#         print('Bin OP : ' )
#         print(node.token)
#         print(node.value)
#         print(node.row)
#         print(node.inverse)
        if node.op.type == AND:
#             print('Node => ')
#             print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index[self.ps.stem(left.row)]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index[self.ps.stem(right.row)]
                right.inverse = False
            
            node.row = intersect_posting(left.row, right.row)
            if node.inverse == True:
                node.row = inverse_posting(self.index, node.row)
                node.inverse = False
            
            return node
        
        elif node.op.type == OR:
#             print('Node => ')
#             print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index([self.ps.stem(left.row)])
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index([self.ps.stem(right.row)])
                right.inverse = False
            

            node.row = union_posting(left.row, right.row)
            if node.inverse == True:
                node.row = inverse_posting(self.index, node.row)
                node.inverse = False
            
            return node
        

    def visit_Num(self, node):
#         print('Num  : ' )
#         print(node.token)
#         print(node.value)
#         print(node.row)
#         print(node.inverse)
        
        node.value = node.value.split('_')[0]
        if self.ps.stem(node.value) in self.index.index.keys():
            
            term_docs = self.index.index[self.ps.stem(node.value)]
            
        else:
            term_docs = {}
#             print('Term Row')
#             print(term_docs)
            
        node.row = term_docs
        if node.inverse == True:
            node.row = inverse_posting(self.index, node.row)
            node.inverse = False
        
        return node

    def interpret(self):
        tree = self.parser.parse()
#         print(tree)
        return self.visit(tree)

def get_boolean_query(query):
    text = str(query)
    text = text.replace(' and ','&')
    text = text.replace(' AND ','&')
    text = text.replace(' or ','|')
    text = text.replace(' OR ','|')
    text = text.replace('NOT', '!')
    text = text.replace('not ','!')
    
    print(text)
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
#     print('Inverted Index')
    
    ps = PorterStemmer()
    lexer = Lexer(text)
    parser = Parser(lexer)
    interpreter = Interpreter(parser, inverted_index, ps)
    result = interpreter.interpret()

#     print(result.value)
#     print(result.row)
    return result.row

def positional_intersect(p1, p2, k):
    
    ip = intersect_posting(p1, p2)
    
    lip = sorted(list(ip.occurrance))
    npl = PostingList()
    ans = []
    
    for doc in lip:
#         print(type(p1))
        positions1 = p1.occurrance[doc]
        positions2 = p2.occurrance[doc]
        index_p2 = 0
        index_p1 = 0
        for pos1 in positions1:
            for pos2 in positions2:
                if pos2['token_no'] -  pos1['token_no'] == k and pos2['token_no'] -  pos1['token_no'] > 0:
                    ans.append({'doc':doc, 'pos1':  pos1, 'pos2':pos2})
                    npl.addOccurrance(doc,pos1)
                    npl.addOccurrance(doc,pos2)
        
        
    return npl
        

def get_phrasal_query(query):
    text = str(query)

    try:
        q1, q2 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Phrasal Query Syntax')
    ps = PorterStemmer()
    q1 = ps.stem(q1)
    q2 = ps.stem(q2)
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
#     print('Inverted Index')
    result = [] 
    p1 = inverted_index.get_term_postings(q1)
    p2 = inverted_index.get_term_postings(q2)
    
    result = positional_intersect(p1, p2, 1)
    
    return result

def get_proximity_query(query):
    text = str(query)
    try:
        q1, q2, q3 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Proximity Query Syntax')
    ps = PorterStemmer()
    q1 = ps.stem(q1)
    q2 = ps.stem(q2)
    k = int(q3[1])+ 1
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
    
    print('Inverted Index')
    result = [] 
    
    p1 = inverted_index.get_term_postings(q1)
    p2 = inverted_index.get_term_postings(q2)
    print(p1)
    result = positional_intersect(p1, p2, k)
    

    return result

## Check Provided Queries:

In [11]:
fails = []

""" Boolean Queries """
q = 'running'
a = {'0', '1', '10', '11', '12', '16', '17', '18', '19', '2', '20', '21', '22', '24', '25', '26', '27', '28', '3', '30', '32', '33', '34', '35', '36', '37', '39', '4', '40', '41', '44', '45', '46', '47', '5', '50', '51', '52', '53', '6', '8', '9'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

q = 'not hammer'
a = {'31', '28', '37', '30', '7', '10', '14', '1', '6', '41', '15', '11', '29', '26', '52', '13', '32', '44', '4', '8', '22', '38', '48', '0', '47', '2', '23', '9', '3', '5', '12', '55'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'actions AND wanted'
a = {'37', '3', '19', '1', '9', '40', '51', '16', '15', '12', '31', '41', '39', '0', '53', '26', '29', '17', '24', '54', '7', '2', '5', '28', '42'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'policies AND western'
a ={'3', '2', '9'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'united OR plane'
a = {'31', '28', '50', '46', '37', '30', '54', '10', '18', '7', '1', '17', '41', '49', '6', '34', '36', '11', '45', '29', '26', '52', '13', '21', '24', '16', '25', '32', '33', '4', '44', '22', '8', '19', '40', '20', '38', '48', '0', '47', '27', '51', '43', '2', '35', '39', '9', '3', '5', '12', '55'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'amazing AND playing AND here'
a = {'51', '8', '0', '13', '39', '18', '19', '41', '20', '40', '16', '21', '50', '11', '27'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'higher AND signals AND enjoyed'
a = {'8'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'higher OR signals OR enjoyed'
a = {'8', '0', '39', '24', '41', '22', '44', '12', '23', '30', '31', '27', '20', '1', '40', '43', '19', '53', '7', '37', '2', '21'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'forgiveness OR developments OR praised'
a = {'3', '8', '18', '20', '32', '43', '9', '17', '36', '44', '52', '42', '37', '38', '16', '31', '5', '2'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'pakistan OR afghanistan OR aid'
a = {'29', '16', '4', '22', '37', '40', '42', '18', '1', '17', '41', '39', '9', '3'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'praised OR ( forgiveness AND developments )'
a = {'36', '42', '44', '2'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'officially OR (warriors AND cartoons)'
a = {'3', '0', '39', '18', '12', '48', '38', '5', '31', '10', '11', '47', '9', '17', '7', '37', '21', '25', '49'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'businessperson AND ( nationwide OR international )'
a = set({})
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'outdated AND ( personnel OR policies)'
a = {'2', '8', '5'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'outdated OR ( personnel AND policies )'
a = {'42', '8', '18', '5', '2', '11'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'biggest AND ( near OR box )'
a = {'18', '46', '54', '43', '4', '45', '50', '53', '47', '6', '51', '44'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'box AND ( united OR year )'
a = {'18', '46', '4', '45', '50', '9', '47', '23', '54', '44', '25'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'biggest AND ( plane OR wanted OR hour)'
a = {'50', '53', '46', '37', '30', '54', '42', '18', '7', '1', '49', '41', '6', '36', '45', '26', '52', '44', '16', '4', '8', '19', '40', '48', '0', '47', '51', '43', '2', '35', '39'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'NOT (united AND plane)'
a = {'31', '28', '50', '53', '46', '37', '54', '42', '7', '10', '14', '18', '6', '49', '41', '15', '11', '45', '13', '21', '44', '16', '4', '8', '22', '40', '20', '38', '48', '47', '51', '43', '23', '39', '9', '3', '5', '12', '55'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'NOT (higher OR signals OR enjoyed)'
a = {'3', '52', '16', '36', '10', '14', '34', '9', '17', '42', '55', '50', '49', '4', '18', '13', '29', '48', '28', '5', '38', '26', '35', '45', '11', '54', '51', '6', '47', '32', '46', '15', '33', '25'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

""" phrasal queries """
q = 'Hillary Clinton' 
a = {'1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

""" positional queries"""
q = 'after years /1'
a = {'6', '7', '44'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'develop solutions /1'
a = {'5', '32'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'keep out /2'
a = {'20', '24', '39', '40', '51'} 
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

' Boolean Queries '

running


'running'

!hammer
total_cnt : 29 docs : [dict_keys([16, 17, 18, 19, 20, 21, 24, 25, 27, 33, 34, 35, 36, 39, 40, 42, 43, 45, 46, 49, 50, 51, 53, 54])]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 26, 28, 29, 30, 31, 32, 37, 38, 41, 44, 47, 48, 52, 55}


'not hammer'

...

'after years /1'

develop solutions /1


'develop solutions /1'

keep out /2


'keep out /2'

In [12]:
# If Result is not Empty, Some Queries are not returning correct docs
list(filter(lambda k : 'FAILED : '+str(list(k.keys)[0]) if len(list(k.values())[0][0]) > 0 else None ,fails))

[]

In [13]:
# Test to check least frequent terms
# m = 100
# val = []
# for p in inverted_index.index:
#     print(p)
#     if len(p) < m:
#         m = len(p)
#         val = p

### Some test queries:

In [14]:
q = input('Enter Boolean Query: e.g outdated OR ( personnel AND policies ) : ')
get_boolean_query(q)
# Output contains posting list

Enter Boolean Query: e.g outdated OR ( personnel AND policies ) : outdated OR ( personnel AND policies )
outdated|( personnel&policies )


total_cnt : 7 docs : [dict_keys([2, 5, 8, 18, 11, 42])]

In [15]:
q = input('Enter phrasal Query: e.g Hillary Clinton')
get_phrasal_query(q)
# Output contains posting list

Enter phrasal Query: e.g Hillary ClintonHillary Clinton


total_cnt : 1116 docs : [dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54])]

In [16]:
q = input('Enter Proximity Query: e.g develop solutions /1 : ')
get_proximity_query(q)
# Output contains posting list

Enter Proximity Query: e.g develop solutions /1 : develop solutions /1
Inverted Index
total_cnt : 33 docs : [dict_keys([2, 3, 5, 8, 9, 16, 17, 18, 20, 31, 32, 37, 38])]


total_cnt : 4 docs : [dict_keys([5, 32])]

In [151]:
# set(inverted_index.docs).symmetric_difference(set(inverted_index.get_term_postings('candidaci').occurrance.keys()))
# set(inverted_index.get_term_postings('candidaci').occurrance.keys())

In [152]:
# set(inverted_index.get_term_postings('candidaci').occurrance.keys())

# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/

https://www.online-utility.org/text/analyzer.jsp

https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex

https://regex101.com/r/M8z3U4/1

https://iq.opengenus.org/porter-stemmer/

https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65

https://ruslanspivak.com/lsbasi-part7/

https://github.com/gintas/django-picklefield